# Parameter sweeps
Parameter sweeps allow you to quickly and easily build a series of related cases that all change one or more aspects of the input model or modeling approximations. Because ARMI automates full-scope engineering analysis, ARMI-driven parameter sweeps are extremely useful for design exploration, sensitivity studies, and statistical analysis. 

To get started with a parameter sweep, you first need some inputs. 

Next, you need an app and a `Case` object as the starting point. 

In [ ]:
# you can only configure an app once
import armi
if not armi.isConfigured():
    armi.configure(armi.apps.App())

In [ ]:
from armi import settings
from armi import cases
from armi.cases import suiteBuilder

cs = settings.Settings('../../../armi/tests/tutorials/anl-afci-177.yaml')
case = cases.Case(cs)

Next, you make a SuiteBuilder, which is the thing that will perturb the input files to generate a suite of related cases from the base case. There are two basic choices, the `FullFactorialSuiteBuilder` which will expand each degree of freedom in every combination (a full multi-dimensional matrix), and the `SeparateEffectsSuiteBuilder` builder, which varies each degree of freedom in isolation. We'll make a FullFactorial case for this demo.

Once you have a `SuiteBuilder`, you start adding one or more degrees of freedom, each of which will adjust one aspect of the input definitions (modeling options, reactor design, etc.).


## A simple one-dimensional parameter sweep

In [ ]:
builder = suiteBuilder.SeparateEffectsSuiteBuilder(case)

Each degree of freedom is defined by an `InputModifier` and a range of values. ARMI contains a few basic `InputModifier` for simple things (like changing settings), and for design-specific param sweeps you can make your own design-specific modifiers. 

The simplest form of parameter sweep just adjusts settings. For example, we could adjust the reactor power from 10 MW to 100 MW in a few steps. 

In [ ]:
import numpy as np

powers = np.linspace(10,100,4)
print(f"Building power modifiers with powers: {powers}")
powerModifications = [suiteBuilder.SettingsModifier('power', mw*1e6) for mw in powers]
builder.addDegreeOfFreedom(powerModifications)
print(f"There are {len(builder.modifierSets)} cases in this suite so far.")

Now we can build the suite. The `Suite` object itself can write input files or just run on the local computer with `suite.run`.

The suite will generate copies of the base case with the power modified across the defined range. 

In [ ]:
suite = builder.buildSuite()
suite.echoConfiguration()

On the other hand, if you want to write inputs and then submit them all to a high-performance computer, you can do that too with `suite.writeInputs()`

In [ ]:
suite.writeInputs()

You can now see that perturbed input files have been produced in the `case-suite` folder.

In [ ]:
!grep -R "power:" case-suite/*

## Modifying the reactor design
Modifying settings is one thing, but the real power of parameter sweeps comes from programatically perturbing the reactor component designs themselves. We accomplish this by modifying ARMI Blueprint objects as derived from the base input. 



In [ ]:
class CladThicknessModifier(suiteBuilder.InputModifier):
    """Modifier that adjust the cladding outer diameter"""
    def __call__(self, cs, blueprints, geom):
        for blockDesign in blueprints.blockDesigns:
            for componentDesign in blockDesign:
                if componentDesign.name == "clad":
                    # by default, values passed to a modifier end up in the 
                    # independentVariable dict
                    componentDesign.od = self.independentVariable["cladThickness"]
                    
cladThicknesses = np.linspace(0.8, 0.9, 5)
builder = suiteBuilder.SeparateEffectsSuiteBuilder(case)
cladModifications = [CladThicknessModifier({"cladThickness":float(od)}) for od in cladThicknesses]
builder.addDegreeOfFreedom(cladModifications)
suite = builder.buildSuite()
suite.echoConfiguration()
suite.writeInputs()

Now we can inspect the input files and see that the cladding outer diameter definition has indeed been modified

In [ ]:
!grep -R "clad:" -A6 case-suite/* | grep "od:"

## A full factorial parameter sweep
Of course, one can use factorial sweeps as well. Below we add two degrees of freedom, one of length 5 and another of length 20. This suite has 100 cases total with all combinations of each setting.

In [ ]:
builder = suiteBuilder.FullFactorialSuiteBuilder(case)
powers = np.linspace(10,100,5)
powerModifications = [suiteBuilder.SettingsModifier('power', mw*1e6) for mw in powers]
builder.addDegreeOfFreedom(powerModifications)

cycleLengths = np.linspace(200,1000,20)
cycleLengthMods = [suiteBuilder.SettingsModifier('cycleLength', mw*1e6) for mw in cycleLengths]
builder.addDegreeOfFreedom(cycleLengthMods)
print(f"There are {len(builder.modifierSets)} cases in this suite.")

## Post-processing the results of the sweep
After all the runs have completed in a parameter sweep, you will want to post-process them to come to some kind of useful conclusion. Because post-processing is very design-specific, you need to make a simple post-processing script. The ARMI framework has useful functions that will assist you in this task. 

First, we assume you're in a new shell and we discover all the cases that ran:

In [ ]:
def loadSuite():
    print('Loading suite results...')
    cs = settings.Settings('../../../armi/tests/tutorials/anl-afci-177.yaml')
    suite = cases.CaseSuite(cs)
    suite.discover(patterns=["anl-afci-177-????.yaml"])
    suite = sorted(suite, key=lambda c: c.cs.inputDirectory)
    return suite
suite = loadSuite()

At this point, you have two options based on your needs:
* Read the ARMI HDF5 output databases directly (useful if you just need to pull certain scalar parameters directly out of the database)
* Have ARMI load HDF5 output databases into full ARMI reactor objects and use the ARMI API to extract data (useful if you want to loop over certain parts of the plant to sum things up)

Directly reading the database will be inherently less stable (e.g. in case the underlying DB format changes), but can be very fast. Loading ARMI reactors for each case is slower, but should also be more powerful and more stable.

After you extract the data, you can plot it or make tables or anything else you need. We often pass it to non-parametric regression systems like the [Alternating Conditional Expectation](https://github.com/partofthething/ace) (ACE) and then on to a multi-objective optimization system (like [Physical Programming](https://github.com/partofthething/physprog)). 